# Uber Rider Data Case Study

December 2016

##  Project overview

Uber is interested in predicting rider retention. To help explore this question, they have provided a sample dataset of a cohort of users who signed up for an account in January 2014. The data was pulled several months later. 

## Dataset description

- city: city this user signed up in
- phone: primary device for this user
- signup_date: date of account registration; in the form ‘YYYY­MM­DD’
- last_trip_date: the last time this user completed a trip; in the form ‘YYYY­MM­DD’ 
- avg_dist: the average distance *(in miles) per trip taken in the first 30 days after signup 
- avg_rating_by_driver: the rider’s average rating over all of their trips 
- avg_rating_of_driver: the rider’s average rating of their drivers over all of their trips 
- surge_pct: the percent of trips taken with surge multiplier > 1
- avg_surge: The average surge multiplier over all of this user’s trips 
- trips_in_first_30_days: the number of trips this user took in the first 30 days after signing up
- luxury_car_user: True if the user took an luxury car in their first 30 days; False otherwise
- weekday_pct: the percent of the user’s trips occurring during a weekday

## Load data and browse data

In [ ]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

% matplotlib inline

In [ ]:
# Load data from file
df = pd.read_csv('data/churn.csv')

In [ ]:
# Inspect dataset
df.info()

In [ ]:
# Browse dataset
df.head(10)

In [ ]:
# Show summary stats
df.describe()

In [ ]:
# Count missing values by column
df.isnull().sum()

## Explore data

### Numeric variables

In [ ]:
df['avg_dist'].plot.hist(bins=20)

In [ ]:
df['avg_surge'].plot.hist(bins=20)

In [ ]:
df['surge_pct'].plot.hist(bins=20)

In [ ]:
df['weekday_pct'].plot.hist(bins=20)

In [ ]:
df['avg_rating_by_driver'].plot.hist(bins=20)

In [ ]:
df['avg_rating_of_driver'].plot.hist(bins=20)

In [ ]:
df['trips_in_first_30_days'].plot.hist(bins=20)

In [ ]:
# # Use scatter_matrix from Pandas
# from pandas.tools.plotting import scatter_matrix
# scatter_matrix(df[[u'avg_dist', u'avg_rating_by_driver', u'avg_rating_of_driver', u'avg_surge', u'surge_pct', u'trips_in_first_30_days', u'weekday_pct']],
#                alpha=0.2, figsize=(16, 16), diagonal='hist')
# plt.show()

In [ ]:
# # Use scatter_matrix from Pandas
# from pandas.tools.plotting import scatter_matrix
# scatter_matrix(df[[u'avg_dist', u'trips_in_first_30_days', u'weekday_pct']], 
#                alpha=0.2, figsize=(8, 8), diagonal='kde')
# plt.show()

### Categorical variables

In [ ]:
df['city'].value_counts()

In [ ]:
df['city'].value_counts().plot.bar()

In [ ]:
df['phone'].value_counts()

In [ ]:
df['phone'].value_counts(dropna=False).plot.bar()

In [ ]:
df['luxury_car_user'].value_counts().plot.bar()

## Clean data - dealing with missing values

In [ ]:
# Count missing values by column
df.isnull().sum()

#### Option 1: drop all rows that have missing values

In [ ]:
df_dropna = df.dropna(axis=0)

In [ ]:
df_dropna.info()

In [ ]:
df_dropna.describe()

#### Option 2: fill missing values

In [ ]:
# Make a copy of df, because you don't want to mess up with orignal df when you experiment stuff
df_fillna = df.copy()

In [ ]:
# Fill missing value for phone
df_fillna['phone'] = df['phone'].fillna('no_phone')

In [ ]:
# Fill missing values with median
df_fillna['avg_rating_by_driver'] = df['avg_rating_by_driver'].fillna(df['avg_rating_by_driver'].median())
df_fillna['avg_rating_of_driver'] = df['avg_rating_of_driver'].fillna(df['avg_rating_of_driver'].median())

In [ ]:
df_fillna.info()

In [ ]:
df_fillna.describe()

#### Decision
We need to decide whether we should exclude data with missing value. We need statistical tools to help us decide. 


In [ ]:
# For now we will move on (to be revisited)
df = df_fillna

## Transform data

### Time-series variables

In [ ]:
# convert time-series information to datetime data type
df['last_trip_date'] = pd.to_datetime(df['last_trip_date'])
df['signup_date'] = pd.to_datetime(df['signup_date'])

In [ ]:
# construct a new df to experiment on the time-series 
df_timestamp = df[['last_trip_date', 'signup_date']].copy()

In [ ]:
df_timestamp['count'] = 1

In [ ]:
df_timestamp = df_timestamp.set_index('signup_date')
df_timestamp['count'].resample("1D").sum().plot()

In [ ]:
df_timestamp = df_timestamp.set_index('last_trip_date')
df_timestamp['count'].resample("1D").sum().plot()

In [ ]:
# Experiment block
date_in_string = '2014-06-01'
date_in_datetime = pd.to_datetime(date_in_string)
print date_in_datetime
print date_in_datetime.dayofweek

In [ ]:
# There might be some signal from day of week when a user signed up Uber, so let's create a column for that
df['signup_dow'] = df['signup_date'].apply(lambda x: x.dayofweek)

In [ ]:
df.head()

### Converting categorical variables

In [ ]:
df.info()

Categorical variables:
* city
* phone
* luxury_car_user
* signup_dow

#### Convert bool columns to int

In [ ]:
df['luxury_car_user'] = df['luxury_car_user'].astype(int)

In [ ]:
df.head()

#### Encode categorical columns to numeric values

In [ ]:
df.head()

In [ ]:
col_category = ['signup_dow', 'city', 'phone']

In [ ]:
df_dummies = pd.get_dummies(df[col_category], columns=col_category)

In [ ]:
df_dummies

In [ ]:
df = df.join(df_dummies)

In [ ]:
df.head()

In [ ]:
df.columns

## Define a label/target/outcome

Add churn indicator. Considered to churn if have not taken a trip in the last 30 days. In practice, you will often have to figure out how to generate a reasonable label to train your dataset. Is the cutoff of 30 days reasonable?  You may want to test this... Sometimes, the correct label is even less obvious; your ability to make a sensible (and defensible) decision in these cases is important.

In [ ]:
# Define churn: users did not take a trip during last 30 days, i.e. last trip date is earlier than 2014-06-01
df['churn'] = (df.last_trip_date < pd.to_datetime('2014-06-01')) * 1
df['active'] = (df.last_trip_date >= pd.to_datetime('2014-06-01')) * 1

df.head()

In [ ]:
df['churn'].mean()

In [ ]:
df['active'].mean()

## EDA with label

### colored scatter_matrix

In [ ]:
colors = ['red' if ix else 'blue' for ix in df['active']]

In [ ]:
# scatter_matrix(df[[u'avg_dist', u'avg_rating_by_driver', u'avg_rating_of_driver', 
#                   u'avg_surge', u'surge_pct', u'trips_in_first_30_days', u'weekday_pct']],
#                alpha=0.2, figsize=(16, 16), diagonal='hist', c=colors)
# plt.show()

### Explore churn rate split by features 

In [ ]:
df[['city', 'churn']].groupby(['city']).mean().plot.bar()

In [ ]:
df[['phone', 'churn']].groupby(['phone']).mean().plot.bar()

In [ ]:
df[['luxury_car_user', 'active']].groupby(['luxury_car_user']).mean().plot.bar()

In [ ]:
df[['trips_in_first_30_days', 'active']].groupby(['active']).mean().plot.bar()

In [ ]:
is_active = df['active'] == 1

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2)
axes[0].hist(df[is_active]['avg_dist'].values)
axes[1].hist(df[~is_active]['avg_dist'].values)
fig.tight_layout()
plt.show()

#### Abstract out the plotting machine

In [ ]:
def hist_active_vs_churn(df, col_name):
    is_active = df['active'] == 1
    fig, axes = plt.subplots(nrows=1, ncols=2)
    axes[0].hist(df[is_active][col_name].values)
    axes[0].set_title("active users")
    axes[0].set_xlabel(col_name)
    axes[0].set_ylabel("counts")
    axes[1].hist(df[~is_active][col_name].values)
    axes[1].set_title("churned users")
    axes[1].set_xlabel(col_name)
    axes[1].set_ylabel("counts")
    fig.tight_layout()
    plt.show()

In [ ]:
df.columns

In [ ]:
hist_active_vs_churn(df, col_name=u'avg_rating_by_driver')

In [ ]:
cols = [u'avg_dist', u'avg_rating_by_driver', u'avg_rating_of_driver', u'avg_surge']

In [ ]:
for col in cols:
    hist_active_vs_churn(df, col_name=col)

## Save cleaned data to csv file

### Select which columns to be saved

In [ ]:
selected_columns = [u'avg_dist', u'avg_rating_by_driver', u'avg_rating_of_driver', u'avg_surge', 
                     u'surge_pct', u'trips_in_first_30_days', u'luxury_car_user', 
                     u'weekday_pct', u'city_Astapor', u'city_King\'s Landing',u'city_Winterfell', 
                     u'phone_Android', u'phone_iPhone', u'phone_no_phone', u'signup_dow_0', 
                     u'signup_dow_1', u'signup_dow_2', u'signup_dow_3', u'signup_dow_4', 
                     u'signup_dow_5', u'signup_dow_6', u'churn']

### Save to csv file


In [ ]:
cleaned_data_csv = 'data/cleaned_data.csv'
df[selected_columns].to_csv(cleaned_data_csv, index=False)

## Build Logistic Regression Model

### Reload data from cleaned csv file

In [ ]:
import pandas as pd
cleaned_data_csv = 'data/cleaned_data.csv'
df = pd.read_csv(cleaned_data_csv)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

### Define Features and Target

In [ ]:
selected_features = [u'avg_dist', u'avg_rating_by_driver', u'avg_rating_of_driver', u'avg_surge', 
                     u'surge_pct', u'trips_in_first_30_days', u'luxury_car_user', 
                     u'weekday_pct', u'city_Astapor', u'city_King\'s Landing',u'city_Winterfell', 
                     u'phone_Android', u'phone_iPhone', u'phone_no_phone', u'signup_dow_0', 
                     u'signup_dow_1', u'signup_dow_2', u'signup_dow_3', u'signup_dow_4', 
                     u'signup_dow_5', u'signup_dow_6']
target = u'churn'

In [ ]:
X = df[selected_features].values
y = df['churn'].values

In [ ]:
X.shape

In [ ]:
y

### Use our own implementation of Logistic Regression Model

In [ ]:
# from my_LogisticRegression import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from my_LogisticRegression import log_likelihood, log_likelihood_gradient, predict, predict_proba
from my_LogisticRegression import GradientAscent
from my_LogisticRegression import precision, accuracy, recall

In [ ]:
ga = GradientAscent(cost=log_likelihood, 
                    gradient=log_likelihood_gradient, 
                    predict_func=predict,
                    fit_intercept=True)
ga.run(X, y, alpha=0.1, num_iterations=2000)

In [ ]:
def plot_cost(cost_history, ax, alpha=1.0):
    """Plot the in sample cost of a gradient ascent run over time."""
    ax.plot(range(len(cost_history)), cost_history, alpha=alpha)
    ax.set_title("Logistic Regression Cost Function Over Time")
    ax.set_xlabel("Iteration Number")
    ax.set_ylabel("Cost")

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 3))

plot_cost(ga.cost_history, ax)

In [ ]:
ga.cost_history[:10]

### Use standardized features

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

In [ ]:
ga = GradientAscent(cost=log_likelihood, 
                    gradient=log_likelihood_gradient, 
                    predict_func=predict,
                    fit_intercept=True)
ga.run(X, y, alpha=0.1, num_iterations=5000)

In [ ]:
fig, ax = plt.subplots(1, figsize=(14, 3))

plot_cost(ga.cost_history, ax)

In [ ]:
y_pred = ga.predict(X)

print("The predicted class vector is \n{}".format(str(y_pred)))
print("The actual class vector is \n{}".format(str(y)))

In [ ]:
print("Accuracy of the Logistic Regression is: {}".format(accuracy(y, y_pred)))
print("Precision of the Logistic Regression is: {}".format(precision(y, y_pred)))
print("Recall of the Logistic Regression is: {}".format(recall(y, y_pred)))

In [ ]:
y.mean()

### Understanding the Estimated Coefficients

In [ ]:
ga.coeffs

In [ ]:
selected_features

In [ ]:
zip(selected_features, ga.coeffs)

In [ ]:
df_coeffs = pd.DataFrame(list(zip(selected_features, ga.coeffs))).sort_values(by=[1], ascending=False)

In [ ]:
df_coeffs.columns = ['feature', 'coeff']
df_coeffs

In [ ]:
import matplotlib.pyplot as plt
% matplotlib inline

In [ ]:
ax = df_coeffs.plot.bar()

In [ ]:
ax = df_coeffs.plot.barh()
t = np.arange(X.shape[1])
ax.set_yticks(t)
ax.set_yticklabels(df_coeffs['feature'])
plt.show()

### How to interpret coefficient?

***Recall: Increasing the value of $x_i$ by 1 increases the odds ratio by a factor of $e^{\beta_i}$***

Say, for a given user, assume he has a probability to churn at 50%, or in another word, the odd ratio is 1:1 = 1

In [ ]:
default_OR = 1 # 50% chance to churn

If a coefficient is 0.2, then, if we increase the corresponding variable by 1 unit, the new odd ratio will be:

In [ ]:
beta = 0.2
increase = np.exp(beta)
OR = default_OR * increase
OR

Which is can be converted to chance to churn:

In [ ]:
p = OR / (1 + OR)
p

If a coefficient is -0.4, then, if we increase the corresponding variable by 1 unit, the new odd ratio will be:

In [ ]:
beta = -0.4
increase = np.exp(beta) * 1
OR = default_OR * increase
OR

Which is can be converted to chance to churn:

In [ ]:
p = OR / (1 + OR)
p

### Check the result with our EDA

In [ ]:
df[['luxury_car_user', 'churn']].groupby(['churn']).mean().plot.bar()

In [ ]:
df[['luxury_car_user', 'churn']].groupby(['luxury_car_user']).mean().plot.bar()

In [ ]:
df[['avg_dist', 'churn']].groupby(['churn']).mean().plot.bar()

In [ ]:
df[['phone_iPhone', 'churn']].groupby(['churn']).mean().plot.bar()

In [ ]:
df[['avg_rating_by_driver', 'churn']].groupby(['churn']).mean().plot.bar()
plt.legend(loc='lower center')

### Use Logistic Regression from sklearn

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=100000, fit_intercept=True)
lr.fit(X, y)

In [ ]:
y_pred = lr.predict(X)
print("Accuracy of the Logistic Regression is: {}".format(accuracy(y, y_pred)))
print("Precision of the Logistic Regression is: {}".format(precision(y, y_pred)))
print("Recall of the Logistic Regression is: {}".format(recall(y, y_pred)))

In [ ]:
df_coeffs = pd.DataFrame(list(zip(selected_features, lr.coef_.flatten()))).sort_values(by=[1], ascending=False)
df_coeffs.columns = ['feature', 'coeff']
df_coeffs

In [ ]:
ax = df_coeffs.plot.barh()
t = np.arange(X.shape[1])
ax.set_yticks(t)
ax.set_yticklabels(df_coeffs['feature'])
plt.show()

### Use polynomial features - high orders!

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

In [ ]:
X_poly = PolynomialFeatures(degree=2, interaction_only=True).fit_transform(X)

In [ ]:
X_poly.shape

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(C=1000000, fit_intercept=True)
lr.fit(X_poly, y)

In [ ]:
y_pred = lr.predict(X_poly)
print("Accuracy of the Logistic Regression is: {}".format(accuracy(y, y_pred)))
print("Precision of the Logistic Regression is: {}".format(precision(y, y_pred)))
print("Recall of the Logistic Regression is: {}".format(recall(y, y_pred)))

### Use train and test set

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [ ]:
print X_train.shape, y_train.shape
print X_test.shape, y_test.shape

In [ ]:
lr = LogisticRegression(C=0.1, fit_intercept=True)
lr.fit(X_train, y_train)

In [ ]:
y_train_pred = lr.predict(X_train)
print("Training score:")
print("Accuracy of the Logistic Regression is: {}".format(accuracy(y_train, y_train_pred)))
print("Precision of the Logistic Regression is: {}".format(precision(y_train, y_train_pred)))
print("Recall of the Logistic Regression is: {}".format(recall(y_train, y_train_pred)))

In [ ]:
y_test_pred = lr.predict(X_test)
print("Test score:")
print("Accuracy of the Logistic Regression is: {}".format(accuracy(y_test, y_test_pred)))
print("Precision of the Logistic Regression is: {}".format(precision(y_test, y_test_pred)))
print("Recall of the Logistic Regression is: {}".format(recall(y_test, y_test_pred)))

### Model Evaluation

#### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test, y_test_pred)

In [ ]:
def plot_confusion_matrix(y_true, y_pred):
    '''
    Code from sklearn example.
    '''
    
    cm = confusion_matrix(y_true, y_pred)

    print(cm)

    # Show confusion matrix in a separate window
    plt.matshow(cm)
    plt.title('Confusion matrix')
    plt.colorbar()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
plot_confusion_matrix(y, y_pred)

#### classification report

In [ ]:
from sklearn.metrics import classification_report
print classification_report(y, y_pred)

In [ ]:
from sklearn.metrics import precision_recall_curve, roc_curve
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score

In [ ]:
lr.predict_proba(X)

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, lr.predict_proba(X_test)[:,1])

In [ ]:
plt.plot(fpr, tpr)

# 45 degree line
xx = np.linspace(0, 1.0, 20)
plt.plot(xx, xx, color='red')

plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC")

plt.show()

In [ ]:
print("Area Under Curve (AUC) of the Logistic Regression is: {}".format(roc_auc_score(y_train, y_train_pred)))

#### Recall-Precision Curve

In [ ]:
prec, rec, thresholds = precision_recall_curve(y_test, lr.predict_proba(X_test)[:,1])

In [ ]:
plt.plot(rec, prec)

plt.xlabel("Recall")
plt.ylabel("Precision")
plt.title("Recall-Precision Curve")

plt.show()

### Profit Curve

In [ ]:
def standard_confusion_matrix(y_true, y_predict):
    [[tn, fp], [fn, tp]] = confusion_matrix(y_true, y_predict)
    return np.array([[tp, fp], [fn, tn]])

def profit_curve(cost_benefit_matrix, probabilities, y_true):
    thresholds = sorted(probabilities, reverse=True)
    profits = []
    for threshold in thresholds:
        y_predict = probabilities > threshold
        confusion_mat = standard_confusion_matrix(y_true, y_predict)
        profit = np.sum(confusion_mat * cost_benefit_matrix) / float(len(y_true))
        profits.append(profit)
    return thresholds, profits

def run_profit_curve(model, costbenefit, X_train, X_test, y_train, y_test):
    probabilities = model.predict_proba(X_test)[:, 1]
    thresholds, profits = profit_curve(costbenefit, probabilities, y_test)
    return thresholds, profits

def plot_profit_model(model, costbenefit, X_train, X_test, y_train, y_test):
    percentages = np.linspace(0, 100, len(y_test))
    thresholds, profits = run_profit_curve(model,
                                           costbenefit,
                                           X_train, X_test,
                                           y_train, y_test)
    plt.plot(percentages, profits, label=model.__class__.__name__)
    plt.title("Profit Curve")
    plt.xlabel("Percentage of test instances (decreasing by score)")
    plt.ylabel("Profit")
    plt.legend(loc='best')
    plt.savefig('profit_curve.png')
    
def find_best_threshold(model, costbenefit, X_train, X_test, y_train, y_test):
    max_threshold = None
    max_profit = None

    thresholds, profits = run_profit_curve(model, costbenefit,
                                           X_train, X_test,
                                           y_train, y_test)
    max_index = np.argmax(profits)
    if profits[max_index] > max_profit:
        max_threshold = thresholds[max_index]
        max_profit = profits[max_index]
    return max_threshold, max_profit

#### Train-test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=42)

#### Build model

In [ ]:
lr = LogisticRegression(C=0.1, fit_intercept=True)
lr.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score

In [ ]:
y_train_pred = lr.predict(X_train)
print("Training score:")
print("Accuracy of the Logistic Regression is: {}".format(accuracy_score(y_train, y_train_pred)))
print("Precision of the Logistic Regression is: {}".format(precision_score(y_train, y_train_pred)))
print("Recall of the Logistic Regression is: {}".format(recall_score(y_train, y_train_pred)))
print("f1-score of the Logistic Regression is: {}".format(f1_score(y_train, y_train_pred)))
print("Area Under Curve (AUC) of the Logistic Regression is: {}".format(roc_auc_score(y_train, y_train_pred)))

In [ ]:
y_test_pred = lr.predict(X_test)
print("Test score:")
print("Accuracy of the Logistic Regression is: {}".format(accuracy_score(y_test, y_test_pred)))
print("Precision of the Logistic Regression is: {}".format(precision_score(y_test, y_test_pred)))
print("Recall of the Logistic Regression is: {}".format(recall_score(y_test, y_test_pred)))
print("f1-score of the Logistic Regression is: {}".format(f1_score(y_test, y_test_pred)))
print("Area Under Curve (AUC) of the Logistic Regression is: {}".format(roc_auc_score(y_test, y_test_pred)))

#### Define cost-benefit matrix based on business input

In [ ]:
costbenefit = np.array([[20, -20], [0, 0]])
costbenefit

#### Plot profit curve 

In [ ]:
plot_profit_model(lr, costbenefit, X_train, X_test, y_train, y_test)

In [ ]:
max_threshold, max_profit = find_best_threshold(lr, costbenefit, X_train, X_test, y_train, y_test)

#### Find the best threshold

In [ ]:
print("The best threshold is {}, which gives a max profit of {}".format(max_threshold, max_profit))

#### Make predictions on new threshold

In [ ]:
y_test_pred = (lr.predict_proba(X_test)[:,1] >= max_threshold).astype(int)
print("Test score:")
print("Accuracy of the Logistic Regression is: {}".format(accuracy_score(y_test, y_test_pred)))
print("Precision of the Logistic Regression is: {}".format(precision_score(y_test, y_test_pred)))
print("Recall of the Logistic Regression is: {}".format(recall_score(y_test, y_test_pred)))
print("f1-score of the Logistic Regression is: {}".format(f1_score(y_test, y_test_pred)))
print("Area Under Curve (AUC) of the Logistic Regression is: {}".format(roc_auc_score(y_test, y_test_pred)))

## More Machine Learning Models

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def print_scores(y_train, y_train_predict, y_test, y_test_predict):
    """
    Print precision, recall, accuracy and f1 scores.
    """
    train_precision = precision_score(y_train, y_train_predict) 
    train_recall = recall_score(y_train, y_train_predict)
    train_accuracy = accuracy_score(y_train, y_train_predict)
    train_f1 = f1_score(y_train, y_train_predict)

    test_precision = precision_score(y_test, y_test_predict)
    test_recall = recall_score(y_test, y_test_predict)
    test_accuracy = accuracy_score(y_test, y_test_predict)
    test_f1 = f1_score(y_test, y_test_predict)

    print "Training score:"
    print train_precision, train_recall, train_accuracy, train_f1
    print "Testing score:"
    print test_precision, test_recall, test_accuracy, test_f1
    
    df_result = pd.DataFrame(data={"Precision": [train_precision, test_precision], 
                                   "Recall": [train_recall, test_recall],
                                   "Accuracy": [train_accuracy, train_f1],
                                   "F1-Score": [train_f1, test_f1]}, 
                             index=["Training Set", "Testing Set"],
                             columns=["Precision", "Recall", "Accuracy", "F1-Score"]
                            )
    return df_result

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

### Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
print_scores(y_train, y_train_predict, y_test, y_test_predict)

### K-Nearest Neighbors


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
print_scores(y_train, y_train_predict, y_test, y_test_predict)

### Bagging


In [ ]:
from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(n_estimators=100)
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
print_scores(y_train, y_train_predict, y_test, y_test_predict)

### Random Forest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=1, n_estimators=100)
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
print_scores(y_train, y_train_predict, y_test, y_test_predict)

In [ ]:
def plot_importance(clf, col_names, max_features=10):
    '''Plot feature importance'''
    feature_importance = clf.feature_importances_
    # make importances relative to max importance
    feature_importance = 100.0 * (feature_importance / feature_importance.max())
    sorted_idx = np.argsort(feature_importance)
    pos = np.arange(sorted_idx.shape[0]) + .5
    
    # Show only top features
    pos = pos[-max_features:]
    feature_importance = (feature_importance[sorted_idx])[-max_features:]
    feature_names = (col_names[sorted_idx])[-max_features:]
    
    plt.barh(pos, feature_importance, align='center')
    plt.yticks(pos, feature_names)
    plt.xlabel('Relative Importance')
    plt.title('Variable Importance')

In [ ]:
plot_importance(clf, df.columns, max_features=10)

In [ ]:
import sklearn.learning_curve as curves
from sklearn.cross_validation import ShuffleSplit

def plt_max_features(X, y):
    """ Calculates the performance of the model as model complexity increases.
        The learning and testing errors rates are then plotted. """
    
    # Create 10 cross-validation sets for training and testing
    cv = ShuffleSplit(X.shape[0], n_iter=10, test_size=0.3, random_state=1)

    # Vary the max_depth parameter from 1 to 10
    features_range = np.arange(1, 12)

    # Create a Random Forest classifier at optimal settings
    clf = RandomForestClassifier(max_depth=8, n_estimators=100, random_state=1)
  
    # Calculate the training and testing scores
    train_scores, test_scores = curves.validation_curve(clf, X, y, 
                                                        param_name='max_features', 
                                                        param_range=features_range, 
                                                        cv=cv, 
                                                        scoring='f1')

    # Find the mean and standard deviation for smoothing
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Plot the validation curve
    plt.figure(figsize=(7, 5))
    plt.title('Random Forest Classifier Complexity Performance')
    plt.plot(features_range, train_mean, 'o-', color = 'r', label = 'Training Score')
    plt.plot(features_range, test_mean, 'o-', color = 'g', label = 'Testing Score')
    plt.fill_between(features_range, train_mean - train_std, train_mean + train_std, alpha=0.15, color='r')
    plt.fill_between(features_range, test_mean - test_std, test_mean + test_std, alpha=0.15, color='g')
    
    # Visual aesthetics
    plt.legend(loc='lower right')
    plt.xlabel('Maximum Features')
    plt.ylabel('F1 Score')
    plt.ylim([0.75,0.90])
    plt.show()

def plt_max_depth(X, y):
    """ Calculates the performance of the model as model complexity increases.
        The learning and testing errors rates are then plotted. """
    
    # Create 10 cross-validation sets for training and testing
    cv = ShuffleSplit(X.shape[0], n_iter=10, test_size=0.3, random_state=1)

    # Vary the max_depth parameter from 1 to 10
    depth_range = np.arange(1, 12)

    # Create a Random Forest classifier at optimal settings
    clf = RandomForestClassifier(max_features=10, n_estimators=100, random_state=1)
  
    # Calculate the training and testing scores
    train_scores, test_scores = curves.validation_curve(clf, X, y,
                                                        param_name='max_depth', 
                                                        param_range=depth_range, 
                                                        cv=cv, 
                                                        scoring='f1')

    # Find the mean and standard deviation for smoothing
    train_mean = np.mean(train_scores, axis=1)
    train_std = np.std(train_scores, axis=1)
    test_mean = np.mean(test_scores, axis=1)
    test_std = np.std(test_scores, axis=1)

    # Plot the validation curve
    plt.figure(figsize=(7, 5))
    plt.title('Random Forest Classifier Complexity Performance')
    plt.plot(depth_range, train_mean, 'o-', color = 'r', label = 'Training Score')
    plt.plot(depth_range, test_mean, 'o-', color = 'g', label = 'Testing Score')
    plt.fill_between(depth_range, train_mean - train_std, train_mean + train_std, alpha = 0.15, color = 'r')
    plt.fill_between(depth_range, test_mean - test_std, test_mean + test_std, alpha = 0.15, color = 'g')
    
    # Visual aesthetics
    plt.legend(loc = 'lower right')
    plt.xlabel('Maximum Depth')
    plt.ylabel('F1 Score')
    plt.ylim([0.60,0.95])
    plt.show()

In [ ]:
plt_max_features(X_train, y_train)

In [ ]:
plt_max_depth(X_train, y_train)

### AdaBoosting


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier(n_estimators=200, learning_rate=1.0)
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
print_scores(y_train, y_train_predict, y_test, y_test_predict)

In [ ]:
plot_importance(clf, df.columns, max_features=10)

In [ ]:
def plot_staged_score(clf, params, X_train, y_train, X_test, y_test):
    '''Plot training deviance. From sklearn documentation'''    
    # compute train set accuracy score    
    train_score = np.array(list(clf.staged_score(X_train, y_train)))
    # compute test set accuracy score    
    test_score = np.array(list(clf.staged_score(X_test, y_test)))

    plt.title('Accuracy')
    plt.plot(np.arange(params['n_estimators']) + 1, train_score, 'b-',
             label='Training Set')
    plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
             label='Test Set')
    plt.legend(loc='lower right')
    plt.xlabel('Boosting Iterations')
    plt.ylabel('Accuracy')

In [ ]:
plot_staged_score(clf, clf.get_params(), X_train, y_train, X_test, y_test)

### Gradient Boosting


In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
clf = GradientBoostingClassifier(n_estimators=500)
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
print_scores(y_train, y_train_predict, y_test, y_test_predict)

In [ ]:
def plot_loss(clf, params, X_test, y_test):
    '''Plot training deviance. From sklearn documentation'''    
    # compute test set deviance
    test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
    
    for i, y_pred in enumerate(clf.staged_decision_function(X_test)):
        test_score[i] = clf.loss_(y_test, y_pred)

    plt.title('Deviance')
    plt.plot(np.arange(params['n_estimators']) + 1, clf.train_score_, 'b-',
             label='Training Set Deviance')
    plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
             label='Test Set Deviance')
    plt.legend(loc='upper right')
    plt.xlabel('Boosting Iterations')
    plt.ylabel(clf.loss)

In [ ]:
plot_loss(clf, clf.get_params(), X_test, y_test)

In [ ]:
plot_importance(clf, df.columns, max_features=10)

### Support Vector Machine


In [ ]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
print_scores(y_train, y_train_predict, y_test, y_test_predict)

#### Since all these algorithms have same sklearn API, we can pack them up and simplify our code

In [ ]:
clf_LRC = LogisticRegression()
clf_KNC = KNeighborsClassifier()
clf_RFC = RandomForestClassifier()
clf_ABC = AdaBoostClassifier()
clf_GBC = GradientBoostingClassifier()
clf_SVC = SVC()

models = [clf_LRC, clf_KNC, clf_RFC, clf_ABC, clf_GBC, clf_SVC]

for clf in models:
    print("\n{}: \n".format(clf.__class__.__name__))
    clf.fit(X_train, y_train)
    y_train_predict = clf.predict(X_train)
    y_predict = clf.predict(X_test)
    print(print_scores(y_train, y_train_predict, y_test, y_predict))

#### We can look at cross validation score

In [ ]:
from sklearn.cross_validation import cross_val_score

clf_LRC = LogisticRegression()
clf_KNC = KNeighborsClassifier()
clf_RFC = RandomForestClassifier()
clf_ABC = AdaBoostClassifier()
clf_GBC = GradientBoostingClassifier()
clf_SVC = SVC()

models = [clf_LRC, clf_KNC, clf_RFC, clf_ABC, clf_GBC, clf_SVC]

for clf in models:
    print "{}:\n".format(clf.__class__.__name__)
    cv_scores = cross_val_score(clf, X, y, cv=5, scoring='f1')
    print("F1-score: %0.2f (+/- %0.2f)" % (cv_scores.mean(), cv_scores.std() * 2))


## Tuning Hyper-Parameters - Grid Search

In [ ]:
from sklearn.grid_search import GridSearchCV

In [ ]:
clf_LRC = LogisticRegression()
clf_KNC = KNeighborsClassifier()
clf_RFC = RandomForestClassifier()
clf_ABC = AdaBoostClassifier()
clf_GBC = GradientBoostingClassifier()
clf_SVC = SVC()

grid_LRC = {
    'penalty': ['l1', 'l2'],
    'class_weight': [None, 'balanced'],
    'C': [1, 10, 100],
}

grid_KNC = {
    'n_neighbors': [5, 10],
    'weights': ['uniform', 'distance'],
    'p': [1, 2]
}

grid_RFC = {
    'criterion': ['gini', 'entropy'],
    'max_features': [None, 'auto', 2, 4, 8],
    'max_depth': [None, 2, 4, 8],
    'n_estimators': [50, 100, 200],
    'class_weight': [None, 'balanced']
}

grid_ABC = {
    'n_estimators': [100, 200, 400],
    'learning_rate': [0.1, 0.5, 1, 5],
}

grid_GBC = {
    'loss': ['deviance', 'exponential'],
    'n_estimators': [100, 200, 400],
    'max_features': [2, 4, 8],    
    'max_depth': [2, 4, 8],
}

grid_SVC =[
    {
        'kernel': ['rbf'], 
        'gamma': [1e-2, 1e-3, 'auto'],
        'C': [0.1, 1, 10]
    },
    {
        'kernel': ['linear'], 
        'C': [0.1, 1, 10]
    },
    {
        'kernel': ['poly'], 
        'degree': [2, 3],
        'C': [0.1, 1, 10]
    }
]

models = [clf_LRC, clf_KNC, clf_RFC, clf_ABC, clf_GBC, clf_SVC]
grids = [grid_LRC, grid_KNC, grid_RFC, grid_ABC, grid_GBC, grid_SVC]

result_models = []

for clf, grid in zip(models, grids):
    print("\n{}: \n".format(clf.__class__.__name__))
    print(grid)
    grid_obj = GridSearchCV(clf, param_grid=grid, scoring='f1', n_jobs=-1)                 
    grid_obj.fit(X_train, y_train)

    result_models.append(grid_obj)
    print(grid_obj.best_estimator_)
    
    y_train_predict = grid_obj.best_estimator_.predict(X_train)
    y_test_predict = grid_obj.best_estimator_.predict(X_test)
    print_scores(y_train, y_train_predict, y_test, y_test_predict)

### Reload data from cleaned csv file

In [ ]:
import pandas as pd
cleaned_data_csv = 'data/cleaned_data.csv'
df = pd.read_csv(cleaned_data_csv)

### Define Features and Target

In [ ]:
selected_features = [u'avg_dist', u'avg_rating_by_driver', u'avg_rating_of_driver', u'avg_surge', 
                     u'surge_pct', u'trips_in_first_30_days', u'luxury_car_user', 
                     u'weekday_pct', u'city_Astapor', u'city_King\'s Landing',u'city_Winterfell', 
                     u'phone_Android', u'phone_iPhone', u'phone_no_phone', u'signup_dow_0', 
                     u'signup_dow_1', u'signup_dow_2', u'signup_dow_3', u'signup_dow_4', 
                     u'signup_dow_5', u'signup_dow_6']
target = u'churn'

In [ ]:
X = df[selected_features].values
y = df['churn'].values

### Define metric scores

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def print_scores(y_train, y_train_predict, y_test, y_test_predict):
    """
    Print precision, recall, accuracy and f1 scores.
    """
    train_precision = precision_score(y_train, y_train_predict) 
    train_recall = recall_score(y_train, y_train_predict)
    train_accuracy = accuracy_score(y_train, y_train_predict)
    train_f1 = f1_score(y_train, y_train_predict)

    test_precision = precision_score(y_test, y_test_predict)
    test_recall = recall_score(y_test, y_test_predict)
    test_accuracy = accuracy_score(y_test, y_test_predict)
    test_f1 = f1_score(y_test, y_test_predict)

    print "Training score:"
    print train_precision, train_recall, train_accuracy, train_f1
    print "Testing score:"
    print test_precision, test_recall, test_accuracy, test_f1
    
    df_result = pd.DataFrame(data={"Precision": [train_precision, test_precision], 
                                   "Recall": [train_recall, test_recall],
                                   "Accuracy": [train_accuracy, train_f1],
                                   "F1-Score": [train_f1, test_f1]}, 
                             index=["Training Set", "Testing Set"],
                             columns=["Precision", "Recall", "Accuracy", "F1-Score"]
                            )
    return df_result

### Standardized features

In [ ]:
from sklearn.preprocessing import StandardScaler
X = StandardScaler().fit_transform(X)

### Train Test split

In [ ]:
from sklearn.cross_validation import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

### Logistic Regression


In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf.fit(X_train, y_train)
y_train_predict = clf.predict(X_train)
y_test_predict = clf.predict(X_test)
print(print_scores(y_train, y_train_predict, y_test, y_test_predict))

### Save model to file

#### convert sklearn model object to pickle object

In [ ]:
import pickle
output_model = pickle.dumps(clf)

#### write pickle object to a file

In [ ]:
filename = 'model.pkl'
with open(filename, 'w') as handle:
    handle.write(output_model)

### Load model from file

#### read pickle object from a file

In [ ]:
filename = 'model.pkl'
with open(filename, 'r') as handle:
    input_model = handle.read()

#### retrieve sklearn model object from pickle object

In [ ]:
clf2 = pickle.loads(input_model)

In [ ]:
print clf2.predict(X[0:10])
print y[:10]


### Pipeline

In [ ]:
import pandas as pd
cleaned_data_csv = 'data/cleaned_data.csv'
df = pd.read_csv(cleaned_data_csv)
selected_features = ['avg_dist', 'avg_rating_by_driver', 'avg_rating_of_driver', 'avg_surge', 
                     'surge_pct', 'trips_in_first_30_days', 'luxury_car_user', 
                     'weekday_pct', 'city_Astapor', 'city_King\'s Landing','city_Winterfell', 
                     'phone_Android', 'phone_iPhone', 'phone_no_phone', 'signup_dow_0', 
                     'signup_dow_1', 'signup_dow_2', 'signup_dow_3', 'signup_dow_4', 
                     'signup_dow_5', 'signup_dow_6']
target = 'churn'
X = df[selected_features].values
y = df['churn'].values

#### Create a pipeline of estimators

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
my_estimators = [('scale', StandardScaler()),
                 ('reduce_dim', PCA()), 
                 ('clf', LogisticRegression())]
my_pipeline = Pipeline(my_estimators)
my_pipeline 


In [ ]:
my_pipeline

In [ ]:
my_pipeline.steps[0][0]

#### fit the pipeline

In [ ]:
my_pipeline.fit(X_train, y_train)

#### make prediction

In [ ]:
y_train_predict = my_pipeline.predict(X_train)
y_test_predict = my_pipeline.predict(X_test)
print(print_scores(y_train, y_train_predict, y_test, y_test_predict))

#### grid search

In [ ]:
from sklearn.grid_search import GridSearchCV
import numpy as np

n_components = [2, 4, 6, 8, 10]
Cs = np.logspace(-4, 4, 3)

#Parameters of pipelines can be set using ‘__’ separated parameter names:

grid_obj = GridSearchCV(my_pipeline,
                        param_grid=dict(reduce_dim__n_components=n_components, clf__C=Cs),
                        scoring='f1', n_jobs=-1
                       )

grid_obj.fit(X_train, y_train)

In [ ]:
grid_obj.best_estimator_

In [ ]:
grid_obj.best_params_

In [ ]:
grid_obj.best_score_